# Proyecto - Asistente de IA para aerolíneas

Ahora, reuniremos lo que hemos aprendido para crear un asistente de IA para atención al cliente para una aerolínea.

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Inicialización

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-jNMIe


In [4]:
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

In [ ]:
# Esta función parece bastante más simple que la de mi video, porque estamos aprovechando las últimas actualizaciones de Gradio.

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Herramientas

Las herramientas son una característica increíblemente poderosa que ofrecen los LLM de vanguardia.

Con las herramientas, puedes escribir una función y hacer que el LLM llame a esa función como parte de su respuesta.

Suena casi espeluznante... ¿le estamos dando el poder de ejecutar código en nuestra máquina?

Bueno, más o menos.

In [43]:
# Vamos a hacer una función útil

ticket_prices = {"londres": "799€", "parís": "899€", "tokyo": "1400€", "berlin": "499€"}

def get_ticket_price(destination_city):
    print(f"Se solicitó la herramienta get_ticket_price para {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def get_wether(destination_city):
    print(f"Se solicitó la herramienta get_wether para {destination_city}")
    wether = {"londres": "lluvia", "parís": "frío", "tokyo": "tifón", "berlin": "nieve"}
    city = destination_city.lower()
    return wether.get(city, "Unknown")

get_wether
Se solicitó la herramienta get_wether para Londres


In [36]:
print(get_ticket_price("berlin"),get_wether("berlin"))

Se solicitó la herramienta get_ticket_price para berlin
Se solicitó la herramienta get_wether para berlin
499€ nieve


In [37]:
# Hay una estructura de diccionario particular que se requiere para describir nuestra función:

price_function = {
    "name": "get_ticket_price",
    "description": "Obtén el precio de un billete de ida y vuelta a la ciudad de destino. Llámalo siempre que necesites saber el precio del billete, por ejemplo, cuando un cliente pregunte '¿Cuánto cuesta un billete a esta ciudad?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

wether_function = {
    "name": "get_wether",
    "description": "Se utiliza cuando se necesite obtener el tiempo de una ciudad, por ejemplo, cuando preguntan ¿Cuál es el tiempo en tokyo? o ¿Habrá buen día en la ciudad?",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad sobre la que se quiere obtener el estado del tiempo",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [38]:
# Y esto está incluido en una lista de herramientas:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": wether_function}
         ]

## Cómo hacer que OpenAI use nuestra herramienta

Hay algunos detalles complicados para permitir que OpenAI "llame a nuestra herramienta"

Lo que realmente hacemos es darle al LLM la oportunidad de informarnos que quiere que ejecutemos la herramienta.

Así es como se ve la nueva función de chat:

In [39]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        # response, city = handle_tool_call(message)
        response = handle_tool_call_01(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [40]:
#Tenemos que escribir esa función handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

def handle_tool_call_01(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    if tool_call.function.name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": message.tool_calls[0].id
        }

    if tool_call.function.name == "get_wether":
        city = arguments.get('destination_city')
        wether = get_wether(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"wether": wether}),
            "tool_call_id": message.tool_calls[0].id
        }

    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)